In [1]:
import json
import building_tokenizer as toker
from brick_parser import pointTagsetList as point_tagsets
import re
import pdb
from functools import reduce
from collections import OrderedDict
from operator import itemgetter

Brick Loaded


In [77]:
building_name = 'ebu3b'
with open('metadata/{0}_label_dict_justseparate.json'.format(building_name)) as fp:
    label_dict = json.load(fp)
with open('metadata/{0}_sentence_dict_justseparate.json'.format(building_name), "r") as fp:
    sentence_dict = json.load(fp)
with open('metadata/{0}_baseline_re.json'.format(building_name), 'r') as fp:
    re_rules = json.load(fp)
with open('metadata/{0}_ground_truth.json'.format(building_name), 'r') as fp:
    truths_dict = json.load(fp)
srcids = label_dict.keys()

lengther = lambda s: len(s[0])
#re_rules = sorted(re_rules, key=lengther, reverse=True)

def adder(x,y):
    return x+y

for srcid in srcids:
    parse_dict = dict()
    labels = label_dict[srcid]
    sentence = ''.join(sentence_dict[srcid])
    found_indices = list()
    for i, re_rule in  enumerate(re_rules):
        find_rule = re_rule[0]
        match_rule = re_rule[1]
        label = re_rule[2]
        found_iter = re.finditer(find_rule, sentence)
        for found in found_iter:
            sub_match = re.search(match_rule, found.group())
            start_idx = found.span()[0]
            try:
                sub_match_pos = sub_match.span()
            except:
                pdb.set_trace()
            pos = (start_idx + sub_match_pos[0], start_idx + sub_match_pos[1])
            one_indices = list(range(*pos))
            if len([True for idx in one_indices if idx in found_indices]) > 0:
                continue
            else:
                found_indices += one_indices
                parse_dict[pos] = label
    # validate indices
    prev_idx = 0
    curr_idx = 0
    wrong_flag = False
    for i, pos in enumerate(sorted(parse_dict.keys())):
        curr_idx = pos[0]
        if curr_idx < prev_idx:
            pdb.set_trace()
        if prev_idx!=curr_idx:
            wrong_flag = True
            break
        else:
            prev_idx = pos[1]
            
    parse_dict = OrderedDict(sorted(parse_dict.items(), key=itemgetter(0)))
    identified = set(parse_dict.values())
    if wrong_flag:
        found_indices = reduce(adder, [list(range(*pos)) for pos in parse_dict.keys()])
        print('----------------------------------------', srcid)
        unfound_cs = [c for i, c in enumerate(sentence) if i not in found_indices]
        print('TRUE: ', truths_dict[srcid])
        print('LBLD: ', identified)
        print('ORIG:', sentence)
        print('LEFT: ', ''.join(unfound_cs))
        print('\n')
    #print m.start(), m.group()

---------------------------------------- 505_3_3015306
TRUE:  ['networkadapter-vnd', 'alarm', 'crac', 'building-ebu3b', 'networkadapter-nae', 'networkadapter-n']
LBLD:  {'none', 'networkadapter-vnd', 'room', 'leftidentifier', 'networkadapter-nae', 'crac', 'networkadapter-n', 'buliding-ebu3b'}
ORIG: nae 05 n2 2 vnd 162 local 1
ebu3b.crac-1.local-1
local alarm 1

LEFT:  locallocallocalala


---------------------------------------- 505_3_3015342
TRUE:  ['networkadapter-vnd', 'crac', 'building-ebu3b', 'networkadapter-nae', 'networkadapter-n', 'high_head_pressure_alarm']
LBLD:  {'none', 'networkadapter-vnd', 'leftidentifier', 'high_head_pressure_alarm', 'networkadapter-nae', 'crac', 'networkadapter-n', 'buliding-ebu3b'}
ORIG: nae 05 n2 2 vnd 163 hi head2
ebu3b.crac-2.hi-head2
high head pressure 2

LEFT:  hiheadhighheadpressure


---------------------------------------- 505_0_3000433
TRUE:  ['networkadapter-vnd', 'current_sensor', 'networkadapter-t', 'hot_water_pump', 'vfd', 'building-ebu3b'